In [1]:
import pandas as pd
from hnmchallenge.data_reader import DataReader
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from hnmchallenge.dataset import Dataset
from hnmchallenge.filtered_dataset import FilterdDataset
from hnmchallenge.stratified_dataset import StratifiedDataset
from hnmchallenge.constant import *
import datetime

In [2]:
import logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.DEBUG)

In [3]:
dataset = StratifiedDataset()
dr = DataReader()

In [4]:
# creating H&M features in holdin 
feature = dataset.get_holdin()

In [5]:
feature

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,0,0,0.008458,2
1,2018-09-20,0,0,0.008458,2
2,2019-08-05,0,0,0.007610,2
3,2019-08-16,0,644,0.039576,2
4,2019-10-06,0,1952,0.030492,2
...,...,...,...,...,...
10955395,2020-09-22,1136204,17478,0.042356,2
10955396,2020-09-22,1136205,10033,0.011847,2
10955397,2020-09-22,1136205,7580,0.022017,2
10955398,2020-09-22,1136205,1765,0.016932,2


In [6]:
feature["tdiff"]=feature['t_dat'].apply(lambda x: 1/(datetime.datetime(2020,9,23) - x).days)

In [7]:
feature["last_buy"] = feature.groupby(DEFAULT_USER_COL)["t_dat"].transform(max)
feature["first_buy"] = feature.groupby(DEFAULT_USER_COL)["t_dat"].transform(min)
feature["time_score"] = ((feature["t_dat"] - feature["first_buy"])/ (feature["last_buy"] - feature["first_buy"])) 


In [8]:
feature["time_score"]=feature["time_score"].fillna(-1)

feature

,t_dat,customer_id,article_id,price,sales_channel_id,tdiff,last_buy,first_buy,time_score
0,2018-09-20,0,0,0.008458,2,0.001362,2020-04-15,2018-09-20,0.000000
1,2018-09-20,0,0,0.008458,2,0.001362,2020-04-15,2018-09-20,0.000000
2,2019-08-05,0,0,0.007610,2,0.002410,2020-04-15,2018-09-20,0.556719
3,2019-08-16,0,644,0.039576,2,0.002475,2020-04-15,2018-09-20,0.575916
4,2019-10-06,0,1952,0.030492,2,0.002833,2020-04-15,2018-09-20,0.664921
...,...,...,...,...,...,...,...,...,...
10955395,2020-09-22,1136204,17478,0.042356,2,1.000000,2020-09-22,2020-09-22,-1.000000
10955396,2020-09-22,1136205,10033,0.011847,2,1.000000,2020-09-22,2020-09-22,-1.000000
10955397,2020-09-22,1136205,7580,0.022017,2,1.000000,2020-09-22,2020-09-22,-1.000000
10955398,2020-09-22,1136205,1765,0.016932,2,1.000000,2020-09-22,2020-09-22,-1.000000


In [9]:
feature.drop(["last_buy", "first_buy"], axis=1)
feature["max_price"] = feature.groupby(DEFAULT_ITEM_COL)["price"].transform("max")
feature["sale_factor"] = (1 - (feature["price"] / feature["max_price"]))

In [10]:
feature

,t_dat,customer_id,article_id,price,sales_channel_id,tdiff,last_buy,first_buy,time_score,max_price,sale_factor
0,2018-09-20,0,0,0.008458,2,0.001362,2020-04-15,2018-09-20,0.000000,0.008458,0.000000
1,2018-09-20,0,0,0.008458,2,0.001362,2020-04-15,2018-09-20,0.000000,0.008458,0.000000
2,2019-08-05,0,0,0.007610,2,0.002410,2020-04-15,2018-09-20,0.556719,0.008458,0.100200
3,2019-08-16,0,644,0.039576,2,0.002475,2020-04-15,2018-09-20,0.575916,0.042356,0.065626
4,2019-10-06,0,1952,0.030492,2,0.002833,2020-04-15,2018-09-20,0.664921,0.030492,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
10955395,2020-09-22,1136204,17478,0.042356,2,1.000000,2020-09-22,2020-09-22,-1.000000,0.042356,0.000000
10955396,2020-09-22,1136205,10033,0.011847,2,1.000000,2020-09-22,2020-09-22,-1.000000,0.050831,0.766922
10955397,2020-09-22,1136205,7580,0.022017,2,1.000000,2020-09-22,2020-09-22,-1.000000,0.076254,0.711269
10955398,2020-09-22,1136205,1765,0.016932,2,1.000000,2020-09-22,2020-09-22,-1.000000,0.042356,0.600240


In [11]:
feature=feature.drop(["last_buy", "first_buy","max_price"], axis=1)

In [12]:
feature

,t_dat,customer_id,article_id,price,sales_channel_id,tdiff,time_score,sale_factor
0,2018-09-20,0,0,0.008458,2,0.001362,0.000000,0.000000
1,2018-09-20,0,0,0.008458,2,0.001362,0.000000,0.000000
2,2019-08-05,0,0,0.007610,2,0.002410,0.556719,0.100200
3,2019-08-16,0,644,0.039576,2,0.002475,0.575916,0.065626
4,2019-10-06,0,1952,0.030492,2,0.002833,0.664921,0.000000
...,...,...,...,...,...,...,...,...
10955395,2020-09-22,1136204,17478,0.042356,2,1.000000,-1.000000,0.000000
10955396,2020-09-22,1136205,10033,0.011847,2,1.000000,-1.000000,0.766922
10955397,2020-09-22,1136205,7580,0.022017,2,1.000000,-1.000000,0.711269
10955398,2020-09-22,1136205,1765,0.016932,2,1.000000,-1.000000,0.600240


In [13]:
dr = DataReader()
feature.to_feather(
        dr.get_preprocessed_data_path() / "stratified_holdin_useritem_dataset.feather"
    )